In [5]:
import opendatasets as od
od.download("https://www.kaggle.com/competitions/histopathologic-cancer-detection")

Skipping, found downloaded files in ".\histopathologic-cancer-detection" (use force=True to force download)


In [6]:
import zipfile
import os

if len(os.listdir("./data")) == 0:
    print("Directory is empty. Zipping file...")
    with zipfile.ZipFile("./histopathologic-cancer-detection/histopathologic-cancer-detection.zip", 'r') as zip_ref:
        zip_ref.extractall("./data")
else:    
    print("Directory is not empty. Skipping zipping!")

Directory is not empty. Skipping zipping!
